In [1]:
import time
import sqlalchemy, os
import sqlite3
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
client = Client(API_KEY, API_SECRET)

db = sqlite3.connect('crypto.db')
db.cursor().execute("""
    CREATE TABLE inventory (
        symbol TEXT,
        avg_price REAL
    )""")
db.commit()
db.close()

### Profit Window (1%)

In [2]:
def fetchall():
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute("SELECT * FROM inventory")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def add_to_db(symbol, price):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"INSERT INTO inventory VALUES ('{symbol}',{price})")
    db.commit()
    db.close()
    
def delete_from_db(symbol):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"DELETE FROM inventory WHERE symbol = '{symbol}'")
    db.commit()
    db.close()

def get_buy_price(symbol):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT avg_price FROM inventory WHERE symbol = '{symbol}'")
    price = cursor.fetchone()[0]
    db.commit()
    db.close()
    return price

def buy_symbol(symbol, price):
    pass

def sell_symbol(symbol):
    pass

In [ ]:
def strategy(list_of_symbols):
    inventory = extract_from_tuple(fetchall())
    for symbol in list_of_symbols:
        if symbol not in inventory:
            try:
                price = float(client.get_avg_price(symbol=symbol)['price'])
            except:
                continue
            add_to_db(symbol,price)
            print(f"{symbol} added to inventory at ${price}")
            
    for symbol in inventory:
        try:
            price = float(client.get_avg_price(symbol=symbol)['price'])
        except:
            continue
        buy_price = get_buy_price(symbol)
        profit = ((price-buy_price)/buy_price)*100
        if profit > 1.01:
            delete_from_db(symbol)
            print(f"Sold {symbol} at ${price} with {profit}% profit")
            
    # print('Total Profit: '+str(total_profit)+'%\n')

def start():
    with open('pairs_level_3.txt','r') as file:
        file.seek(0) # Ensure you're at the start of the file..
        first_char = file.read(1) # Get the first character
        if not first_char:
            u=0 # The first character is the empty string..
        else:
            file.seek(0) # The first character wasn't empty. Return to the start of the file.
             # Use file now
            strategy(file.readline().split(','))

print("Program starts")

while(True):
    start()

Program starts
CAKEUSDT added to inventory at $4.77098209
Sold POWRUSDT at $0.3277615 with 2.63307943327485% profit
Sold BELUSDT at $1.07372472 with 6.812901639022509% profit
BELUSDT added to inventory at $1.07372472
Sold CAKEUSDT at $4.82300991 with 1.090505456078945% profit
CAKEUSDT added to inventory at $4.82384018
Sold BELUSDT at $1.08472084 with 1.0241097923124973% profit
BELUSDT added to inventory at $1.08513697
Sold BELUSDT at $1.0961284 with 1.012907154015776% profit
BELUSDT added to inventory at $1.09623232
Sold BELUSDT at $1.10736192 with 1.0152592472369426% profit
BELUSDT added to inventory at $1.10747178
